# Predicting incorrect student answers from DataShop data
<p style="margin:30px">
    <img width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
</p>

In this tutorial, we show how to predict whether a student will succesfully answer a problem using a dataset from [CMU DataShop](https://pslcdatashop.web.cmu.edu/). While online courses are logistically efficient, the structure can make it more difficult for a teacher to understand how students are learning in their class. To try to fill in those gaps, we can apply machine learning. However, building an accurate machine learning model requires extracting information called **features**. Finding the right features is a crucial component of both finding a satisfactory answer and of interpreting the dataset as a whole. The process of **feature engineering** is made simple by [Featuretools](http://www.featuretools.com).

*If you're running this notebook yourself, please download the [geometry dataset](https://pslcdatashop.web.cmu.edu/DatasetInfo?datasetId=76) into the `data` folder in this repository. You can follow this [instructions](https://pslcdatashop.web.cmu.edu/help?datasetId=76&page=export) to download the data. You will only need the `.txt` file. The infrastructure in this notebook will work with **any** DataShop dataset, but you will need to change the file name in the first cell.*

## Highlights
* Show how to import a DataShop dataset into featuretools
* Demonstrate efficacy of automatic feature generation by training a machine learning model 
* Give an example of how Featuretools can reveal and help answer interesting questions

# Step 1: Load data
At the beginning of any project, it is worthwhile to take a moment to think about how your dataset is structured.

In these datasets the unique events come from `transactions`: places where a student interacts with a system. Each transaction has a `time-index`, the time at which information in a row becomes known. Furthermore, the columns of those transactions have variables that can be grouped together. 

For instance, there are only 59 distinct students for the 6778 transactions we have in the geometry dataset. Those students log in to the system and have individual sessions. We can break down problems and problem steps in a similar way.

Featuretools stores data in an `EntitySet`. This is an abstraction which allows us to hold on to not only the data itself, but also to metadata like relationships and column types.

We create an entityset structure using the `datashop_to_entityset` function in [utils](utils.py). If you're interested in how `datashop_to_entityset` is structured, there's an associated notebook [entityset_function](entityset_function.ipynb) which explains choices made in more detail.

In [1]:
# Note that each branch is a one -> many relationship

# schools       students     problems
#        \        |         /
#   classes   sessions   problem steps
#          \     |       /
#           transactions
#

import utils

filename = 'data/ds76_tx_All_Data_74_2018_0912_070949.txt'
es = utils.datashop_to_entityset(filename)
es

Entityset: Dataset
  Entities:
    problem_steps [Rows: 78, Columns: 156]
    sessions [Rows: 59, Columns: 3]
    students [Rows: 59, Columns: 2]
    transactions [Rows: 6778, Columns: 26]
    problems [Rows: 20, Columns: 2]
    classes [Rows: 1, Columns: 2]
    schools [Rows: 1, Columns: 1]
  Relationships:
    transactions.Step Name -> problem_steps.Step Name
    problem_steps.Problem Name -> problems.Problem Name
    transactions.Session Id -> sessions.Session Id
    sessions.Anon Student Id -> students.Anon Student Id
    transactions.Class -> classes.Class
    classes.School -> schools.School

Our `students` entity represents that: there are only 59 rows, one for each Anonymous student ID.

In [2]:
es['students'].df.head(3)

,Anon Student Id,first_sessions_time
Stu_c0bf45c22dc46067350d304ce330067e,Stu_c0bf45c22dc46067350d304ce330067e,1996-02-01 00:00:00
Stu_af3a2f63bda8c1338556108cb8d519a0,Stu_af3a2f63bda8c1338556108cb8d519a0,1996-02-01 00:00:02
Stu_d7f18a5fa205a889b0c5b0b56a7127d3,Stu_d7f18a5fa205a889b0c5b0b56a7127d3,1996-02-01 00:00:02


Featuretools allows us to make new entities as grouped by categorical values. Through this process of *normalization* we have created 8 connected entities from an initial table of transactions. We can look at what is left in `transactions` after normalization.

In [3]:
es['transactions'].df.head(3)

,Transaction Id,Sample Name,Total Num Hints,Level (Unit),Tutor Response Subtype,Time Zone,Input,Problem View,Feedback Classification,Class,...,End Time,Time,Action,Outcome,Step Name,Student Response Type,Student Response Subtype,Tutor Response Type,Is Last Attempt,Duration (sec)
bdc63de3ac6ace889eed3850997ea333,bdc63de3ac6ace889eed3850997ea333,All Data,NaN,Area,NaN,US/Eastern,NaN,1,NaN,NaN,...,1996-02-01 00:00:00,1996-02-01 00:00:00,NaN,0,(CIRCLE-AREA_A QUESTION1),ATTEMPT,NaN,RESULT,0,0
34f5d8fcf207513d222bf6d1bced2046,34f5d8fcf207513d222bf6d1bced2046,All Data,NaN,Area,NaN,US/Eastern,NaN,1,NaN,NaN,...,1996-02-01 00:00:02,1996-02-01 00:00:01,NaN,1,(CIRCLE-AREA_A QUESTION1),ATTEMPT,NaN,RESULT,1,1
171f36289815d9782f8beab6ad8b0fac,171f36289815d9782f8beab6ad8b0fac,All Data,NaN,Area,NaN,US/Eastern,NaN,1,NaN,NaN,...,1996-02-01 00:00:02,1996-02-01 00:00:02,NaN,0,(AREA QUESTION1),ATTEMPT,NaN,RESULT,0,0


# Step 2: Building Features
Next, we calculate a feature matrix on the `transactions` entity to try to predict the outcome of a given transaction. It's at this step that our previous setup pays off: we can automatically calculate features using data from the whole `EntitySet`. 

## Cutoff times
We are going to be generating features and doing predictive modeling on time-sensitive data. That comes with a high risk of label leakage. 

In this case, we are predicting if a student will get a particular problem correct. For a fixed problem, the feature "There exists an attempt number three" would be highly predictive of the result on attempts one and two. There can only be a third attempt if there first two attempts were wrong! In that way, storing future `attempt` information in a feature to predict `Outcome` would yield higher test accuracy than the model deserves. It's not ok to have the feature "There exists an attempt number three" while predicting attempts one and two because it contains information that can not be known at that point in time.

To circumvent that, we introduce the notion of [cutoff_times](https://docs.featuretools.com/automated_feature_engineering/handling_time.html). A `cutoff_time` has an index column and a datetime column indicating the last acceptable date we can use while generating features for a historical training example. We can also add in a label, which will be passed through Deep Feature Synthesis ([DFS](https://docs.featuretools.com/automated_feature_engineering/afe.html)) untouched so we can recover it later.

Setting cutoff times immediately mitigates the risk of fraudulently using future data, controls the number of predictions we make and controls what data is used while calculating features.


In [4]:
cutoff_times = es['transactions'].df[['Transaction Id', 'End Time', 'Outcome']]
cutoff_times.head()

,Transaction Id,End Time,Outcome
bdc63de3ac6ace889eed3850997ea333,bdc63de3ac6ace889eed3850997ea333,1996-02-01 00:00:00,0
34f5d8fcf207513d222bf6d1bced2046,34f5d8fcf207513d222bf6d1bced2046,1996-02-01 00:00:02,1
171f36289815d9782f8beab6ad8b0fac,171f36289815d9782f8beab6ad8b0fac,1996-02-01 00:00:02,0
5ad4ed1a55851edb89fc23d8ed4076e1,5ad4ed1a55851edb89fc23d8ed4076e1,1996-02-01 00:00:02,1
de0bfeea6b32cb37891c0b7dbc6b7a18,de0bfeea6b32cb37891c0b7dbc6b7a18,1996-02-01 00:00:02,1


With that in hand, we can guarentee that future values for `Outcome` won't be used for any calculations because we set the time index of `Outcome` to be after the cutoff time.

From there, we can call `ft.dfs` to generate our features and feature matrix. Deep Feature Synthesis creates features using reusable functions ([Primitives](https://docs.featuretools.com/automated_feature_engineering/primitives.html)). The algorithm attempts to combine primitives together with actual data to create a feature matrix. Here, we'll use the primitives `Sum`, `Mean`, `PercentTrue` and `Hour`. On an ordinary laptop, you should expect the following cell to take roughly 40 minutes to complete as there are more than 3000 unique cutoff times. For faster results, uncomment the approximate line. 

In [5]:
import featuretools as ft
from featuretools.selection import remove_low_information_features

import pandas as pd
import numpy as np
pd.options.display.max_columns = 500

fm, features = ft.dfs(entityset=es,
                      target_entity='transactions',
                      agg_primitives=['Sum', 'Mean', 'Percent_True'],
                      trans_primitives=['Hour'],
                      max_depth=3,
                      # approximate='2m',
                      cutoff_time=cutoff_times[1000:],
                      verbose=True)

# Encode the feature matrix using One-Hot encoding
fm_enc, f_enc = ft.encode_features(fm, features)
fm_enc = fm_enc.fillna(0)
fm_enc = remove_low_information_features(fm_enc)

# Pop the label
label = fm_enc.pop('Outcome')

fm.tail()

Built 464 features
Elapsed: 47:11:55 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks  


,Input,Selection,Tutor Response Type,Attempt At Step,Tutor Response Subtype,Problem View,Level (Unit),Time Zone,Help Level,Class,Student Response Type,Sample Name,Session Id,Step Name,Total Num Hints,Feedback Classification,Feedback Text,Student Response Subtype,Action,problem_steps.KC Category (hLFASearchModel1-back-context).1,problem_steps.KC Category (Area),problem_steps.KC (Lasso Model).1,problem_steps.CF (Factor repeat),problem_steps.KC (hLFASearchAICWholeModel3arith),problem_steps.KC (hLFASearchModel1-renamed-chgd2),problem_steps.KC Category (Geometry),problem_steps.KC Category (Lasso Model).4,problem_steps.KC (LuS-divide-compose-by-addition),problem_steps.KC Category (LFASearchAIC2_with_texkbook_new_decompose),problem_steps.KC Category (Orige-trap-merge),problem_steps.KC Category (LFASearchAICWholeModel2),problem_steps.KC (Textbook_New_Decompose),problem_steps.KC Category (hLFASearchModel1_context-single),problem_steps.KC (hLFASearchModel1-context).1,problem_steps.KC Category (hLFASearchModel1-back-context).2,problem_steps.KC (Textbook),problem_steps.KC (Item),problem_steps.CF (Factor add-or-m),problem_steps.KC Category (Single_Plus_Unique),problem_steps.KC Category (hLFASearchModel1-renamed-chgd2),problem_steps.KC (combineTraps),problem_steps.KC (Merge-Trap),problem_steps.KC Category (Textbook_New_Decompose),HOUR(End Time),problem_steps.KC (Concepts),problem_steps.KC Category (hLFASearchModel1-back-context),problem_steps.KC Category (Lasso Model).2,problem_steps.KC (Single_Plus_Unique),problem_steps.KC (xDecmpTrapCheat),problem_steps.KC (DecompArithDiam),problem_steps.KC (hLFASearchAICWholeModel3arith0),problem_steps.KC Category (hLFASearchModel1-backward).1,problem_steps.KC Category (Concepts),problem_steps.KC Category (WESST),problem_steps.KC Category (new1),problem_steps.KC Category (DecomposeArith),problem_steps.KC (Yu-Ju_Textbook_modified),problem_steps.KC Category (Orig-trap-merge),problem_steps.CF (Factor basic-shape),problem_steps.KC Category (hLFASearchModel1-context).1,problem_steps.KC Category (new),problem_steps.KC (Single_Plus_Unique).1,problem_steps.KC Category (hLFASearchAICWholeModel3arith2),problem_steps.KC (Circle-Collapse),problem_steps.KC Category (Original-test),HOUR(Problem Start Time),problem_steps.KC Category (Lasso Model),problem_steps.KC Category (Merge-Trap),problem_steps.KC Category (DecompArithDiam2),problem_steps.KC Category (xDecmpTrapCheat),problem_steps.CF (Factor required),problem_steps.KC (LFASearchAIC1_with_texkbook_new_decompose),problem_steps.KC Category (Item Model),problem_steps.KC Category (Original),problem_steps.KC (Lasso Model).4,problem_steps.KC (Lasso Model).5,problem_steps.KC (Geometry),problem_steps.KC (Orige-trap-merge),problem_steps.KC (WESST),problem_steps.KC Category (hLFASearchModel1-renamed-chgd),problem_steps.KC Category (Decompose),classes.School,problem_steps.KC (new KC model name),problem_steps.CF (Factor parallelogram-type),problem_steps.KC Category (new KC model name),problem_steps.CF (Factor trapezoid-part),problem_steps.KC Category (Lasso Model).5,problem_steps.CF (Factor embeddedness),problem_steps.KC Category (Circle-Collapse),problem_steps.KC (hLFASearchModel1_context-single),problem_steps.KC (Lasso Model),problem_steps.Problem Name,problem_steps.KC (DecompArithDiam2),problem_steps.KC (Original),problem_steps.KC (JohnsNewModel),problem_steps.KC Category (Lasso Model).1,problem_steps.KC (LFASearchAIC2_no_textbook_new_decompose),problem_steps.KC (hLFASearchModel1-context),problem_steps.KC (hLFASearchModel1-renamed-chgd),problem_steps.CF (Factor backward),problem_steps.KC Category (Yu-Ju_Textbook_modified2),problem_steps.KC (Original-test),problem_steps.KC Category (hLFASearchAICWholeModel3arith0),problem_steps.KC Category (JohnsNewModel),problem_steps.KC (Lasso Model).2,problem_steps.KC Category (Yu-Ju_Textbook_modified),problem_steps.KC Category (Lasso Model).3,problem_steps.KC (combineTraps-diffSize),problem_steps.KC Category (Textbook),problem_steps.KC (

Above, you can scroll to the right to see the 227 features we created. If you look at the column names, you can see that we've done more than individually apply primitives one at a time to the raw data. Features were stacked and combined across entities in an exhaustive way. Using Deep Feature Synthesis is powerful because it greatly increases the likelihood of finding important features while decreasing the workload of the data scientist.

# Step 3: Making predictions

Here we split the data into two parts using `train_test_split` from scikit-learn. Notice that we don't want the splitter to shuffle our data, since that has the risk to leak labels in time sensitive data. 

We can do feature selection with [Recursive Feature Elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html). This recursively removes features by checking feature importances (according to some model) with smaller and smaller feature sets. Here we'll set `RFE` to select 20 features.  

In [6]:
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from utils import feature_importances
from sklearn.feature_selection import RFE

# 1. Split X and y into a train and test set
X_train, X_test, y_train, y_test = train_test_split(fm_enc, label, shuffle=False)

# 2. Select features using RFE
clf = RandomForestClassifier()
estimator = clf
selector = RFE(estimator, 20, step=1)
selector = selector.fit(X_train, y_train)
X_train.iloc[:, selector.support_].tail()

/Users/lecrintech/projects/python/demos-featuretools/demo6/predict-correct-answer/env2.7/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,Attempt At Step = 2,problem_steps.MEAN(transactions.Is Last Attempt),sessions.MEAN(transactions.Is Last Attempt),sessions.PERCENT_TRUE(transactions.Outcome),sessions.MEAN(transactions.Duration (sec)),problem_steps.MEAN(transactions.Duration (sec)),problem_steps.SUM(transactions.Duration (sec)),sessions.SUM(transactions.Duration (sec)),problem_steps.SUM(transactions.Problem View),problem_steps.PERCENT_TRUE(transactions.Outcome),sessions.students.MEAN(transactions.Duration (sec)),sessions.students.MEAN(transactions.Is Last Attempt),problem_steps.problems.PERCENT_TRUE(transactions.Outcome),sessions.students.MEAN(transactions.Problem View),problem_steps.problems.SUM(transactions.Problem View),sessions.students.SUM(transactions.Duration (sec)),problem_steps.problems.MEAN(transactions.Is Last Attempt),sessions.students.PERCENT_TRUE(transactions.Outcome),problem_steps.problems.MEAN(transactions.Duration (sec)),problem_steps.problems.SUM(transactions.Duration (sec))
Transaction Id,,,,,,,,,,,,,,,,,,,,
39eae7f8590d2b934315064e9855afff,0,0.444444,0.360656,0.767347,8.024590,7.571429,477.0,1958.0,74,0.609375,8.024590,0.360656,0.702128,1.440816,160,1958.0,0.507143,0.767347,8.850000,1239.0
882b1884a9ae8720e1dc00747889797d,0,0.750000,0.359712,0.744681,9.978417,7.500000,1290.0,1387.0,181,0.832370,9.978417,0.359712,0.850610,1.787234,345,1387.0,0.773006,0.744681,9.800613,3195.0
b2fb7c0e5010305b5af967a99c21ae1d,0,0.619048,0.502732,0.820652,9.295082,10.785714,453.0,1701.0,47,0.837209,9.295082,0.502732,0.829457,1.929348,279,1701.0,0.580392,0.820652,9.850980,2512.0
cf0d3d46691ec06d7c3d4f522d1a8e83,1,0.425963,0.437500,0.753846,13.593750,4.351927,4291.0,1740.0,1486,0.765182,13.593750,0.437500,0.770965,1.907692,1625,1740.0,0.435411,0.753846,5.274616,5839.0
ccd74208da111b48054439b5eca33dca,1,0.627907,0.344398,0.744856,8.078838,13.139535,565.0,1947.0,51,0.822222,8.078838,0.344398,0.833333,1.205761,279,1947.0,0.582031,0.744856,9.855469,2523.0


Finally, we can train a Random Forest Classifier to make predictions. Those predictions can be checked against our `y_test` from above, and scored with a [roc_auc_score](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). Below we'll train and score our model and output the five most important features according to this model.

In [7]:
# 3. Train a Random Forest Classifier
clf.fit(selector.transform(X_train), y_train)

# 4. Make predictions and score
probs = clf.predict_proba(selector.transform(X_test))
print("Auc score of {:.3f}".format(roc_auc_score(y_test, probs[:,1])))

feats = feature_importances(X_train.iloc[:, selector.support_], clf)

Auc score of 0.646
Feature Importances: 
1: Attempt At Step = 2
2: problem_steps.PERCENT_TRUE(transactions.Outcome)
3: problem_steps.problems.PERCENT_TRUE(transactions.Outcome)
4: sessions.PERCENT_TRUE(transactions.Outcome)
5: problem_steps.MEAN(transactions.Duration (sec))
-----



Let's examine a feature. The feature `problem_steps.MEAN(transactions.Duration (sec))` is the average time spent on a given problem step. It's easy to see how 'amount of time people spend on this problem' might be related to problem difficulty and ultimately the `Outcome` of a given attempt.

# Next Steps
This notebook showed how to structure your data and make predictions with machine learning. Rather than spending time creating features, it's now possible to explore the relationships and implications betweem thousands of features directly. Reasonable next steps might be to:
1. Try [plotting](#Appendix:-Plotting) some of the generated features
2. Run feature selection and tune the machine learning model
3. Explore other prediction problems on this `EntitySet`




# Appendix: Plotting
Here, we'll look at a couple of important features as created above. We can use plots to help us understand why certain automatically generated features are good. Here, we'll plot two important features from the model above and match results from the model to our own intuition.

In [8]:
from bokeh.io import show, output_notebook, output_file

output_notebook()
output_file('difficulty_vs_time.html')

p = utils.datashop_plot(fm,
                        col1='problem_steps.problems.PERCENT_TRUE(transactions.Outcome)',
                        col2='problem_steps.problems.MEAN(transactions.Duration (sec))',
                        label=label,
                        names=['Problem difficulty versus problem time', 
                               'Success rate on this problem', 
                               'Average time on this problem'])
show(p)


Loading BokehJS ...

![](data/images/exampleimage.png)

If you're interested in understanding particular points and clusters [click here](https://www.featuretools.com/wp-content/uploads/2018/03/difficulty_vs_time.html) for an interactive html version. That version will allow you to zoom in and hover over individual points to see which problem step and problem it is. 

Notice that while a feature like *Success rate on this problem* might only have one value if we use all of the data, the graph here shows that data changing with time. To start our analysis, let's get a baseline for the data. The blue dots represent a successful answer while the grey dots indicate an incorrect answer. We can ask how often students are correct on average:

In [9]:
print('Overall success rate is {:.2f}%'.format(100 * np.mean(fm['Outcome'])))

Overall success rate is 79.39%


That is, if you were to pick a point at random, there's a roughly 79% chance it will be a correct answer. There are sections of this graph where that sample is more likely to be correct, and more likely to be incorrect, which can be picked up by the decision trees that make up our Random Forest. From the graph it looks like there is a spike of correct answers in this dataset near problems that take 10 seconds. We can verify that directly:

In [10]:
maxtime = 15
duration_feat = fm['problem_steps.problems.MEAN(transactions.Duration (sec))']
problem_feat = fm['problem_steps.problems.PERCENT_TRUE(transactions.Outcome)']

print('If problem takes more than {} seconds: {:.2f}% of problems answered correctly'.format(maxtime,
    100 * np.mean(fm[(duration_feat >= maxtime)]['Outcome'])))

If problem takes more than 15 seconds: 74.20% of problems answered correctly


In other words, the average time spent on a problem is an indicator of whether or not a student will answer the problem correctly in this dataset. There are a number of possible interpretations and testable hypotheses associated to that. It is clear that the averages don't tell the whole story of what's going on. Let's look at success rate as sorted by problem.



In [11]:
split_line = .85

print('If Success Rate > {}: {:.2f}% of problems answered correctly'.format(split_line,
    100 * np.mean(fm[(problem_feat >= split_line)]['Outcome'])))

If Success Rate > 0.85: 88.07% of problems answered correctly


In [12]:
print('Problems with higher success than {}:'.format(split_line))
for f in fm[problem_feat >= split_line]['problem_steps.Problem Name'].unique():
    print(f)

Problems with higher success than 0.85:
BUILDING_A_SIDEWALK
RECTANGLE_ABCD
POGS
PAINTING_THE_WALL
CIRCLE_O
DESIGNING_A_QUILT


That is, of the 20 problems in this data set, only 6 have a success rate that was over 85% at some point in time. In that way the machine learning has indicated that how previous students have done on the problem is a good predictor of how they will do inside of this dataset.

In addition to our earlier conclusion that "the problems that took a long time had worse scores", we have a secondary conclusion that "some problems are harder than others". What makes this line of inquiry neat is that we didn't have to do very much work to reveal interesting questions. In that way we have used automated feature engineering to make explicit our implicit understanding of this dataset.

In [13]:
# Save output files

import os

try:
    os.mkdir("output")
except:
    pass

fm.to_csv('output/feature_matrix.csv')
cutoff_times[1000:].to_csv('output/cutoff_times.csv')

<p>
    <img src="https://www.featurelabs.com/wp-content/uploads/2017/12/logo.png" alt="Featuretools" />
</p>

Featuretools was created by the developers at [Feature Labs](https://www.featurelabs.com/). If building impactful data science pipelines is important to you or your business, please [get in touch](https://www.featurelabs.com/contact/).